In [0]:
import pyspark
from pyspark.sql import SparkSession


In [0]:
spark=SparkSession.builder.appName("LogisticRegression").getOrCreate()

In [0]:
###Logistic Regression

In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
input_data=spark.read.csv("dbfs:/FileStore/shared_uploads/purvajainpj123@gmail.com/customer_churn.csv",inferSchema=True,header=True)

In [0]:
input_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [0]:
input_data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [0]:
input_data.count()

Out[79]: 900

In [0]:
test_data=spark.read.csv("dbfs:/FileStore/shared_uploads/purvajainpj123@gmail.com/new_customers.csv",inferSchema=True,header=True)

In [0]:
test_data.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

In [0]:
test_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [0]:
from pyspark.ml.feature import VectorAssembler
assembler=VectorAssembler(inputCols=["Age","Total_Purchase","Account_Manager","Years","Num_Sites"],outputCol="features")

In [0]:
output_data=assembler.transform(input_data)

In [0]:
output_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
output_data.head(1)

Out[86]: [Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1, features=DenseVector([42.0, 11066.8, 0.0, 7.22, 8.0]))]

In [0]:
final_data=output_data.select("features","Churn")

In [0]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Churn: integer (nullable = true)



In [0]:
train,test=final_data.randomSplit([0.7,0.3])

In [0]:
model=LogisticRegression(labelCol="Churn")

In [0]:
model=model.fit(train)

In [0]:
summary=model.summary

In [0]:
summary.predictions.describe().show()

+-------+-------------------+------------------+
|summary|              Churn|        prediction|
+-------+-------------------+------------------+
|  count|                629|               629|
|   mean|0.17965023847376788|0.1478537360890302|
| stddev| 0.3842013866418944|0.3552374341532129|
|    min|                0.0|               0.0|
|    max|                1.0|               1.0|
+-------+-------------------+------------------+



In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
predictions=model.evaluate(test)

In [0]:
predictions.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[5.22318487755756...|[0.99463876294997...|       0.0|
|[25.0,9672.03,0.0...|    0|[5.39559029784035...|[0.99548394557172...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.75618274222248...|[0.99984254017544...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.73629115254687...|[0.85021535994374...|       0.0|
|[29.0,5900.78,1.0...|    0|[4.60308489149848...|[0.99007854693573...|       0.0|
|[29.0,9617.59,0.0...|    0|[5.07374540915643...|[0.99377999667269...|       0.0|
|[29.0,11274.46,1....|    0|[4.96668503979246...|[0.99308198977464...|       0.0|
|[29.0,13255.05,1....|    0|[4.58264365408651...|[0.98987572755185...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.55583588369630...|[0.99858022471205...|       0.0|
|[30.0,8677.28,1

In [0]:
evaluator=BinaryClassificationEvaluator(rawPredictionCol="prediction",labelCol="Churn")

In [0]:
evaluator.evaluate(predictions.predictions)

Out[98]: 0.6927119427119427

In [0]:
model1=LogisticRegression(labelCol="Churn")
model1=model1.fit(final_data)

In [0]:
test_data=assembler.transform(test_data)

In [0]:
results=model1.transform(test_data)

In [0]:
results.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|[2.22168680572546...|[0.90218015921764...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|[-6.2207539991845...|[0.00198380259784...|       

In [0]:
results.select("company","prediction").show()

+----------------+----------+
|         company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

